# Features By Userid (Median and STD of TransactionAmt)

In [3]:
import pandas as pd
train = pd.read_parquet('../../input/train.parquet')
test = pd.read_parquet('../../input/test.parquet')
userids = pd.read_csv('../../data/userids/userids-001.csv')
tt = pd.concat([train, test], axis=0, sort=False)
tt = tt.merge(userids, left_index=True, right_on=['TransactionID'])

tt_small = tt[['userid', 'TransactionAmt']].copy()
tt_small = tt.reset_index()[['userid', 'TransactionAmt', 'TransactionID']].copy()
tt_small = pd.merge(tt_small,
         tt_small[['userid', 'TransactionAmt']].groupby('userid').median().rename(columns={'TransactionAmt':'TransactionAmt_userid_median'}),
         on='userid',
         how='left',
         validate="m:1")

tt_small = pd.merge(tt_small,
         tt_small[['userid', 'TransactionAmt']].groupby('userid').std().rename(columns={'TransactionAmt':'TransactionAmt_userid_std'}),
         on='userid',
         how='left',
         validate="m:1")

In [42]:
tr006 = pd.read_parquet('../../data/train_FE006.parquet')
te006 = pd.read_parquet('../../data/test_FE006.parquet')

In [46]:
tr007 = tr006.merge(tt_small[['TransactionID','TransactionAmt_userid_median','TransactionAmt_userid_std']], on='TransactionID', validate='1:1')
te007 = te006.merge(tt_small[['TransactionID','TransactionAmt_userid_median','TransactionAmt_userid_std']], on='TransactionID', validate='1:1')

In [47]:
tr007.to_parquet('../../data/train_FE007.parquet')
te007.to_parquet('../../data/test_FE007.parquet')

In [4]:
import gc

In [5]:
del tt
gc.collect()

2262

In [8]:
tt_small['TransactionAmt_expanding_userid_median'] = tt_small.groupby('userid')['TransactionAmt'].expanding().median()

KeyboardInterrupt: 

In [7]:
tt_small.dtypes

userid                            int64
TransactionAmt                  float64
TransactionID                     int64
TransactionAmt_userid_median    float64
TransactionAmt_userid_std       float64
dtype: object

In [9]:
tmp = tt_small.groupby('userid')['TransactionAmt'].expanding().median().reset_index(drop=False).drop('userid',axis=1)
tmp = tmp.set_index(tmp.columns[0]) # Set the index so we can rejoin back
# How different is your transactionAmt compared to historical?
tt_small['TransactionAmt_expanding_userid_median'] = tmp.astype('float32')

In [11]:
%%time
tmp = tt_small.groupby('userid')['TransactionAmt'].expanding().std().reset_index(drop=False).drop('userid',axis=1)
tmp = tmp.set_index(tmp.columns[0]) # Set the index so we can rejoin back
# How different is your transactionAmt compared to historical?
tt_small['TransactionAmt_expanding_userid_std'] = tmp.astype('float32')

CPU times: user 7min 46s, sys: 8.86 s, total: 7min 55s
Wall time: 7min 44s


In [12]:
%%time
tt_small['TransactionAmt_expanding_userid_median'] = tt_small.groupby('userid')['TransactionAmt'].expanding().median()

TypeError: incompatible index of inserted column with frame index